In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import accuracy_score
from keras.layers import LSTM, Dense,Input,Layer
from keras import backend as K,Model
from numpy.random import seed
from imblearn.under_sampling import RandomUnderSampler 
seed(0)
import tensorflow as tf
tf.random.set_seed(0)

In [2]:
data = pd.read_csv('creditcard.csv', na_filter=True)

data.drop(['Time'], axis=1, inplace=True)
selected_vars =['V4','V14','V10','V16','V9','V12','V11','V17','V7','V21','V18','V8','V1','V3','V6','V2','V27','V5','V26','Class']


In [3]:
tr_data = data[selected_vars]
tr_data.shape

(284807, 20)

In [4]:
X = tr_data.drop(['Class'], axis = 'columns')
Label_Data = tr_data['Class']

In [5]:
y=tr_data['Class']

In [6]:
from sklearn.preprocessing import StandardScaler
## Standardizing the data
X=pd.DataFrame(StandardScaler().fit_transform(X))

In [7]:
X_train0,X_test,y_train0,y_test = train_test_split(X, y, test_size=0.3, shuffle=False,random_state=0)
X_train0,X_train4,y_train0,y_train4 = train_test_split(X_train0, y_train0, test_size=0.20, shuffle=False,random_state=0)
X_train0,X_train3,y_train0,y_train3 = train_test_split(X_train0, y_train0, test_size=0.25, shuffle=False,random_state=0)
X_train0,X_train2,y_train0,y_train2 = train_test_split(X_train0, y_train0, test_size=0.333, shuffle=False,random_state=0)
X_train0,X_train1,y_train0,y_train1 = train_test_split(X_train0, y_train0, test_size=0.50, shuffle=False,random_state=0)

from sklearn.linear_model import SGDOneClassSVM
df1_normal=X_train0[y_train0==0]
clf = SGDOneClassSVM(nu=0.03,random_state=0)
res = clf.fit_predict(df1_normal)
res[res==1] = 0
res[res==-1] = 1
ros = RandomUnderSampler(sampling_strategy=0.95)
ros.fit_resample(df1_normal,res)
index=ros.sample_indices_
X_train0.drop(X_train0.index[index], inplace=True)
y_train0.drop(y_train0.index[index], inplace=True)

df1_normal=X_train1[y_train1==1]
clf = SGDOneClassSVM(nu=0.06,random_state=0)
res = clf.fit_predict(df1_normal)
res[res==1] = 0
res[res==-1] = 1
ros = RandomUnderSampler(sampling_strategy=0.95)
ros.fit_resample(df1_normal,res)
index=ros.sample_indices_
X_train1.drop(X_train1.index[index], inplace=True)
y_train1.drop(y_train1.index[index], inplace=True)

df1_normal=X_train2[y_train2==1]
clf = SGDOneClassSVM(nu=0.09,random_state=0)
res = clf.fit_predict(df1_normal)
res[res==1] = 0
res[res==-1] = 1
ros = RandomUnderSampler(sampling_strategy=0.95)
ros.fit_resample(df1_normal,res)
index=ros.sample_indices_
X_train2.drop(X_train2.index[index], inplace=True)
y_train2.drop(y_train2.index[index], inplace=True)


df1_normal=X_train3[y_train3==1]
clf = SGDOneClassSVM(nu=0.1,random_state=0)
res = clf.fit_predict(df1_normal)
res[res==1] = 0
res[res==-1] = 1
ros = RandomUnderSampler(sampling_strategy=0.95)
ros.fit_resample(df1_normal,res)
index=ros.sample_indices_
X_train3.drop(X_train3.index[index], inplace=True)
y_train3.drop(y_train3.index[index], inplace=True)

df1_normal=X_train4[y_train4==1]
clf = SGDOneClassSVM(nu=0.11,random_state=0)
res = clf.fit_predict(df1_normal)
res[res==1] = 0
res[res==-1] = 1
ros = RandomUnderSampler(sampling_strategy=0.95)
ros.fit_resample(df1_normal,res)
index=ros.sample_indices_
X_train4.drop(X_train4.index[index], inplace=True)
y_train4.drop(y_train4.index[index], inplace=True)

In [8]:
X_train0.shape,X_train1.shape,X_train2.shape,X_train3.shape,X_train4.shape

((37183, 19), (39889, 19), (39829, 19), (39867, 19), (39871, 19))

In [9]:
y_train0.shape,y_train1.shape

((37183,), (39889,))

In [10]:
X_test.shape

(85443, 19)

# N=1

In [11]:
# X_train et X_test sont des dataframe qui contient les features
train_LSTM_X0=X_train0.to_numpy()
test_LSTM_X=X_test.to_numpy()
train_LSTM_y0=y_train0.to_numpy()
test_LSTM_y=y_test.to_numpy()
w=3
X_train0=np.zeros([train_LSTM_X0.shape[0]-w+1,w, train_LSTM_X0.shape[1]])
for i in range(train_LSTM_X0.shape[0]-w+1):
    for j in range(w):
        X_train0[i][j]=train_LSTM_X0[i+j]
X_test=np.zeros([test_LSTM_X.shape[0]-w+1,w, test_LSTM_X.shape[1]])
for i in range(test_LSTM_X.shape[0]-w+1):
    for j in range(w):
        X_test[i][j]=test_LSTM_X[i+j]
## Recuperation des labels

y_train0=np.delete(train_LSTM_y0,[*range(0, w-1, 1)], 0)
y_test=np.delete(test_LSTM_y,[*range(0, w-1, 1)], 0)

train_LSTM_X0=X_train0
test_LSTM_X=X_test
train_LSTM_y0=y_train0
test_LSTM_y=y_test

In [12]:
inputs=Input((w,19))
x1=LSTM(50)(inputs)
outputs=Dense(1,activation='sigmoid')(x1)
model0=Model(inputs,outputs)

In [13]:
model0.compile(loss='binary_crossentropy', optimizer='adam', metrics=['Precision','Recall'])

In [14]:
history0=model0.fit(train_LSTM_X0, train_LSTM_y0,epochs=4,batch_size=100,validation_data=(test_LSTM_X, test_LSTM_y))

Epoch 1/4
372/372 [==============================] - 7s 12ms/step - loss: 0.0897 - precision: 0.0339 - recall: 0.2451 - val_loss: 0.0906 - val_precision: 0.0687 - val_recall: 0.4630
Epoch 2/4
372/372 [==============================] - 4s 10ms/step - loss: 0.0056 - precision: 0.7320 - recall: 0.6961 - val_loss: 0.0319 - val_precision: 0.7582 - val_recall: 0.6389
Epoch 3/4
372/372 [==============================] - 4s 10ms/step - loss: 0.0045 - precision: 0.7573 - recall: 0.7647 - val_loss: 0.0180 - val_precision: 0.9432 - val_recall: 0.7685
Epoch 4/4
372/372 [==============================] - 4s 10ms/step - loss: 0.0039 - precision: 0.7477 - recall: 0.8137 - val_loss: 0.0116 - val_precision: 0.9432 - val_recall: 0.7685


In [15]:
# predict probabilities for test set
yhat_probs0 = model0.predict(test_LSTM_X, verbose=0)
# reduce to 1d array
yhat_probs0 = yhat_probs0[:, 0]> 0.5

In [16]:
# demonstration of calculating metrics for a neural network model using sklearn
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(test_LSTM_y, yhat_probs0)
print('Accuracy: %f' % accuracy)
# precision tp / (tp + fp)
precision = precision_score(test_LSTM_y, yhat_probs0)
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(test_LSTM_y, yhat_probs0)
print('Recall: %f' % recall)
f1 = f1_score(test_LSTM_y, yhat_probs0)
print('f1: %f' % f1)

Accuracy: 0.999649
Precision: 0.943182
Recall: 0.768519
f1: 0.846939


# N=2

In [17]:
# X_train et X_test sont des dataframe qui contient les features
train_LSTM_X1=X_train1.to_numpy()
train_LSTM_y1=y_train1.to_numpy()
w=3
X_train1=np.zeros([train_LSTM_X1.shape[0]-w+1,w, train_LSTM_X1.shape[1]])
for i in range(train_LSTM_X1.shape[0]-w+1):
    for j in range(w):
        X_train1[i][j]=train_LSTM_X1[i+j]

y_train1=np.delete(train_LSTM_y1,[*range(0, w-1, 1)], 0)
train_LSTM_X1=X_train1
train_LSTM_y1=y_train1

In [18]:
inputs=Input((w,19))
x1=LSTM(50)(inputs)
outputs=Dense(1,activation='sigmoid')(x1)
model1=Model(inputs,outputs)

In [19]:
model1.compile(loss='binary_crossentropy', optimizer='adam', metrics=['Precision','Recall'])

In [20]:
train_LSTM_X1.shape,train_LSTM_X0.shape

((39887, 3, 19), (37181, 3, 19))

In [21]:
history1=model1.fit(train_LSTM_X1, train_LSTM_y1,epochs=6,batch_size=100,validation_data=(test_LSTM_X, test_LSTM_y))

Epoch 1/6
399/399 [==============================] - 7s 12ms/step - loss: 0.0744 - precision: 0.0608 - recall: 0.2697 - val_loss: 0.0998 - val_precision: 0.0412 - val_recall: 0.5556
Epoch 2/6
399/399 [==============================] - 4s 9ms/step - loss: 0.0046 - precision: 0.9538 - recall: 0.6966 - val_loss: 0.0407 - val_precision: 0.5532 - val_recall: 0.7222
Epoch 3/6
399/399 [==============================] - 4s 10ms/step - loss: 0.0037 - precision: 0.9605 - recall: 0.8202 - val_loss: 0.0205 - val_precision: 0.8817 - val_recall: 0.7593
Epoch 4/6
399/399 [==============================] - 4s 11ms/step - loss: 0.0033 - precision: 0.9367 - recall: 0.8315 - val_loss: 0.0126 - val_precision: 0.9213 - val_recall: 0.7593
Epoch 5/6
399/399 [==============================] - 4s 10ms/step - loss: 0.0030 - precision: 0.9610 - recall: 0.8315 - val_loss: 0.0074 - val_precision: 0.9425 - val_recall: 0.7593
Epoch 6/6
399/399 [==============================] - 4s 11ms/step - loss: 0.0027 - precisio

In [22]:
# predict probabilities for test set
yhat_probs1 = model1.predict(test_LSTM_X, verbose=0)
# reduce to 1d array
yhat_probs1 = yhat_probs1[:, 0]> 0.5

In [23]:
# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(test_LSTM_y, yhat_probs1)
print('Accuracy: %f' % accuracy)
# precision tp / (tp + fp)
precision = precision_score(test_LSTM_y, yhat_probs1)
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(test_LSTM_y, yhat_probs1)
print('Recall: %f' % recall)
f1 = f1_score(test_LSTM_y, yhat_probs1)
print('f1: %f' % f1)

Accuracy: 0.999637
Precision: 0.952941
Recall: 0.750000
f1: 0.839378


# N=3

In [24]:
# X_train et X_test sont des dataframe qui contient les features
train_LSTM_X2=X_train2.to_numpy()
train_LSTM_y2=y_train2.to_numpy()
w=3
X_train2=np.zeros([train_LSTM_X2.shape[0]-w+1,w, train_LSTM_X2.shape[1]])
for i in range(train_LSTM_X2.shape[0]-w+1):
    for j in range(w):
        X_train2[i][j]=train_LSTM_X2[i+j]

y_train2=np.delete(train_LSTM_y2,[*range(0, w-1, 1)], 0)
train_LSTM_X2=X_train2
train_LSTM_y2=y_train2

In [25]:
inputs=Input((w,19))
x1=LSTM(50)(inputs)
outputs=Dense(1,activation='sigmoid')(x1)
model2=Model(inputs,outputs)

In [26]:
model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['Precision','Recall'])

In [27]:
history2=model2.fit(train_LSTM_X2, train_LSTM_y2,epochs=5,batch_size=100,validation_data=(test_LSTM_X, test_LSTM_y))

Epoch 1/5
399/399 [==============================] - 8s 11ms/step - loss: 0.0716 - precision: 0.0052 - recall: 0.0196 - val_loss: 0.1033 - val_precision: 0.0076 - val_recall: 0.1111
Epoch 2/5
399/399 [==============================] - 4s 9ms/step - loss: 0.0045 - precision: 0.9412 - recall: 0.3137 - val_loss: 0.0517 - val_precision: 0.1514 - val_recall: 0.5185
Epoch 3/5
399/399 [==============================] - 4s 9ms/step - loss: 0.0035 - precision: 0.9697 - recall: 0.6275 - val_loss: 0.0315 - val_precision: 0.4825 - val_recall: 0.6389
Epoch 4/5
399/399 [==============================] - 4s 11ms/step - loss: 0.0032 - precision: 0.9474 - recall: 0.7059 - val_loss: 0.0247 - val_precision: 0.6404 - val_recall: 0.6759
Epoch 5/5
399/399 [==============================] - 4s 11ms/step - loss: 0.0030 - precision: 0.9231 - recall: 0.7059 - val_loss: 0.0167 - val_precision: 0.7723 - val_recall: 0.7222


In [28]:
# predict probabilities for test set
yhat_probs2 = model2.predict(test_LSTM_X, verbose=0)
# reduce to 1d array
yhat_probs2 = yhat_probs2[:, 0]> 0.5

In [29]:
# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(test_LSTM_y, yhat_probs2)
print('Accuracy: %f' % accuracy)
# precision tp / (tp + fp)
precision = precision_score(test_LSTM_y, yhat_probs2)
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(test_LSTM_y, yhat_probs2)
print('Recall: %f' % recall)
f1 = f1_score(test_LSTM_y, yhat_probs2)
print('f1: %f' % f1)

Accuracy: 0.999380
Precision: 0.772277
Recall: 0.722222
f1: 0.746411


# N=4

In [30]:
# X_train et X_test sont des dataframe qui contient les features
train_LSTM_X3=X_train3.to_numpy()
train_LSTM_y3=y_train3.to_numpy()
w=3
X_train3=np.zeros([train_LSTM_X3.shape[0]-w+1,w, train_LSTM_X3.shape[1]])
for i in range(train_LSTM_X3.shape[0]-w+1):
    for j in range(w):
        X_train3[i][j]=train_LSTM_X3[i+j]

y_train3=np.delete(train_LSTM_y3,[*range(0, w-1, 1)], 0)
train_LSTM_X3=X_train3
train_LSTM_y3=y_train3

In [31]:
inputs=Input((w,19))
x1=LSTM(50)(inputs)
outputs=Dense(1,activation='sigmoid')(x1)
model3=Model(inputs,outputs)

In [32]:
model3.compile(loss='binary_crossentropy', optimizer='adam', metrics=['Precision','Recall'])

In [33]:
history3=model3.fit(train_LSTM_X3, train_LSTM_y3,epochs=12,batch_size=100,validation_data=(test_LSTM_X, test_LSTM_y))

Epoch 1/12
399/399 [==============================] - 8s 12ms/step - loss: 0.0874 - precision: 0.0634 - recall: 0.4054 - val_loss: 0.0070 - val_precision: 1.0000 - val_recall: 0.1111
Epoch 2/12
399/399 [==============================] - 4s 10ms/step - loss: 0.0057 - precision: 0.9487 - recall: 0.6667 - val_loss: 0.0045 - val_precision: 0.9811 - val_recall: 0.4815
Epoch 3/12
399/399 [==============================] - 4s 10ms/step - loss: 0.0046 - precision: 0.9205 - recall: 0.7297 - val_loss: 0.0039 - val_precision: 0.9692 - val_recall: 0.5833
Epoch 4/12
399/399 [==============================] - 4s 10ms/step - loss: 0.0041 - precision: 0.9348 - recall: 0.7748 - val_loss: 0.0032 - val_precision: 0.9848 - val_recall: 0.6019
Epoch 5/12
399/399 [==============================] - 4s 10ms/step - loss: 0.0037 - precision: 0.9551 - recall: 0.7658 - val_loss: 0.0030 - val_precision: 0.9867 - val_recall: 0.6852
Epoch 6/12
399/399 [==============================] - 4s 10ms/step - loss: 0.0033 - p

In [34]:
# predict probabilities for test set
yhat_probs3 = model3.predict(test_LSTM_X, verbose=0)
# reduce to 1d array
yhat_probs3 = yhat_probs3[:, 0]> 0.5

In [35]:
# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(test_LSTM_y, yhat_probs3)
print('Accuracy: %f' % accuracy)
# precision tp / (tp + fp)
precision = precision_score(test_LSTM_y, yhat_probs3)
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(test_LSTM_y, yhat_probs3)
print('Recall: %f' % recall)
f1 = f1_score(test_LSTM_y, yhat_probs3)
print('f1: %f' % f1)

Accuracy: 0.999614
Precision: 0.987013
Recall: 0.703704
f1: 0.821622


# N=5

In [36]:
# X_train et X_test sont des dataframe qui contient les features
train_LSTM_X4=X_train4.to_numpy()
train_LSTM_y4=y_train4.to_numpy()
w=3
X_train4=np.zeros([train_LSTM_X4.shape[0]-w+1,w, train_LSTM_X4.shape[1]])
for i in range(train_LSTM_X4.shape[0]-w+1):
    for j in range(w):
        X_train4[i][j]=train_LSTM_X4[i+j]

y_train4=np.delete(train_LSTM_y4,[*range(0, w-1, 1)], 0)
train_LSTM_X4=X_train4
train_LSTM_y4=y_train4

In [37]:
inputs=Input((w,19))
x1=LSTM(50)(inputs)
outputs=Dense(1,activation='sigmoid')(x1)
model4=Model(inputs,outputs)

In [38]:
model4.compile(loss='binary_crossentropy', optimizer='adam', metrics=['Precision','Recall'])

In [39]:
history4=model4.fit(train_LSTM_X4, train_LSTM_y4,epochs=12,batch_size=200,validation_data=(test_LSTM_X, test_LSTM_y))

Epoch 1/12
200/200 [==============================] - 6s 15ms/step - loss: 0.1352 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: 0.0107 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/12
200/200 [==============================] - 2s 11ms/step - loss: 0.0056 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: 0.0064 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 3/12
200/200 [==============================] - 2s 11ms/step - loss: 0.0037 - precision: 1.0000 - recall: 0.0357 - val_loss: 0.0048 - val_precision: 1.0000 - val_recall: 0.1204
Epoch 4/12
200/200 [==============================] - 2s 10ms/step - loss: 0.0029 - precision: 0.8571 - recall: 0.2143 - val_loss: 0.0039 - val_precision: 1.0000 - val_recall: 0.4259
Epoch 5/12
200/200 [==============================] - 2s 11ms/step - loss: 0.0024 - precision: 0.9286 - recall: 0.4643 - val_loss: 0.0035 - val_precision: 1.0000 - val_recall: 0.5556
Epoch 6/12
200/200 [==============================] -

In [40]:
# predict probabilities for test set
yhat_probs4 = model4.predict(test_LSTM_X, verbose=0)
# reduce to 1d array
yhat_probs4 = yhat_probs4[:, 0]> 0.5

In [41]:
# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(test_LSTM_y, yhat_probs4)
print('Accuracy: %f' % accuracy)
# precision tp / (tp + fp)
precision = precision_score(test_LSTM_y, yhat_probs4)
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(test_LSTM_y, yhat_probs4)
print('Recall: %f' % recall)
f1 = f1_score(test_LSTM_y, yhat_probs4)
print('f1: %f' % f1)

Accuracy: 0.999637
Precision: 1.000000
Recall: 0.712963
f1: 0.832432


# FFNN

In [42]:
y_probs0 = model0.predict(train_LSTM_X0, verbose=0)
y_probs1 = model1.predict(train_LSTM_X0, verbose=0)
y_probs2 = model2.predict(train_LSTM_X0, verbose=0)
y_probs3 = model3.predict(train_LSTM_X0, verbose=0)
y_probs4 = model3.predict(train_LSTM_X0, verbose=0)

y_probs00 = model0.predict(train_LSTM_X1, verbose=0)
y_probs11 = model1.predict(train_LSTM_X1, verbose=0)
y_probs22 = model2.predict(train_LSTM_X1, verbose=0)
y_probs33 = model3.predict(train_LSTM_X1, verbose=0)
y_probs44 = model4.predict(train_LSTM_X1, verbose=0)

In [43]:
X_train0=np.concatenate((y_probs0,y_probs1,y_probs2,y_probs3,y_probs4),axis=1)
X_train1=np.concatenate((y_probs00,y_probs11,y_probs22,y_probs33,y_probs44),axis=1)
X_train=np.concatenate((X_train0,X_train1),axis=0)
y_train=np.concatenate((y_train0,y_train1),axis=0)
X_train.shape,y_train.shape

((77068, 5), (77068,))

In [44]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier().fit(X_train, y_train)

# evaluation

In [45]:
yhat_probs0 = model0.predict(test_LSTM_X, verbose=0)
yhat_probs1 = model1.predict(test_LSTM_X,verbose=0)
yhat_probs2 = model2.predict(test_LSTM_X,verbose=0)
yhat_probs3 = model3.predict(test_LSTM_X,verbose=0)
yhat_probs4 = model3.predict(test_LSTM_X,verbose=0)
X_test=np.concatenate((yhat_probs0,yhat_probs1,yhat_probs2,yhat_probs3,yhat_probs4),axis=1)
y_pred=clf.predict(X_test)

In [46]:
# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(test_LSTM_y, y_pred)
print('Accuracy: %f' % accuracy)
# precision tp / (tp + fp)
precision = precision_score(test_LSTM_y, y_pred)
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(test_LSTM_y, y_pred)
print('Recall: %f' % recall)
#F1 = 2 * (precision * recall) / (precision + recall)
f1=f1_score(test_LSTM_y, y_pred)
print('f1: %f' % f1)

Accuracy: 0.999625
Precision: 0.931818
Recall: 0.759259
f1: 0.836735
